In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import MeCab
import ipadic

In [2]:
import sys

sys.path.append('/home/jovyan/mi-drive/medinfo_lab/Research_Projects/zhang/Husky')

In [3]:
from tools.tokenizers import myMeCab

In [4]:
!ls /opt/conda/lib/python3.10/site-packages/ipadic/

dicdir	dicdir-O  __init__.py  ipadic.py  __pycache__


In [5]:
TokenizerPath = '/home/jovyan/mi-drive/medinfo_lab/Research_Projects/zhang/Husky/kuhpTokenizer/40000_vocab/bert_bpe_vocab_40000.model'
CorpusPath = '/home/jovyan/mi-drive/medinfo_lab/Research_Projects/zhang/Husky/data/del_none_data_for_train_tokenizer.txt'

In [6]:
def load_data(CorpusPath):
    with open(CorpusPath, 'r', encoding='utf-8') as f:
        data = f.readlines()
    return data

In [7]:
data = load_data(CorpusPath)

In [8]:
def tokenize(data):
    tokenized_data = []
    for line in data:
        _tokens = myMeCab.tokenize(text=line, stemmer=False)
        tokenized_data.extend(_tokens)
    return tokenized_data

In [9]:
from joblib import Parallel, delayed

In [10]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True, tokenizer=tokenize)

In [11]:
def parallelize_vectorization(text):
    return tfidf_vectorizer.fit_transform([text])

In [ ]:
multi_processing = Parallel(n_jobs=2)
tfidf_matrix = multi_processing(delayed(parallelize_vectorization)(text) for text in data)

In [ ]:
from tools.valuations.entropy import calculate_entropy

In [ ]:
output = []
sentence_matrix = tfidf_vectorizer.transform(data[:1000]).toarray()
for index, line in enumerate(data[:1000]):
    ens = calculate_entropy(sentence_matrix, batch=True)
    output.append(ens[index], line)

In [ ]:
import matplotlib.pyplot as plt
entropy = []
for item in output:
    entropy.append(item[0])
plt.hist(entropy, bins=80)
plt.xlabel('entropy')
plt.ylabel('count')
plt.title('entropy distribute Plot')
plt.savefig('example_plot.png')
plt.show()